# This notebook is for fairness and diversity validation for our ADS

## Fairness validation
- Focus on the group fairness validation considering sensitive attribute gender, age and whether-smoking.
- Group fairness methodology includes 3 measures proposed in [Measuring Fairness in Ranked Outputs](https://arxiv.org/abs/1610.08559).
- Validation is done on different groups defined by above sensitive attributes.
- We also validate the cross impact between multiple sensitive attributes.

## Diversity
- Diversity component shows the representation of different groups defined by the above sensitve attributes.
- We show the representations for top-10 and over-all results.